In [1]:
# Load packages
# If you need to install packages, please do not leave the output of installation in your homework submission.
using CSV, DataFrames, CategoricalArrays, Plots, Statistics, Random, StatsPlots, JuMP, DataFrames, Random


In [11]:
df = CSV.read("PCA_df.csv", DataFrame)
first(df, 5)


Row,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,1.25902,1.07441,-0.0870012,-0.0666432,-0.4028,-0.214531,-0.115619,-0.204249,-0.0865234,-0.126472,-0.0529494,-0.187418,-0.33505,0.22008,0.106203,0.0541629,0.014138,-0.0180455,-0.0715133,-0.0156755,0.179807,-0.190544,0.327443,-0.00827776,0.177347,-0.0582593,-0.0627241,-0.09792,-0.0982399,-0.0377172,-0.0369587,-0.0381895,-0.110458,0.120058,0.00224206,0.235474,0.0601376,-0.0583418,-0.0288259,0.0313048,0.00749979,0.122497,0.0341467,0.144343,0.0757099,0.0698748,-0.0699144,-0.01949,-0.0971503,-0.0809212,-0.374342,0.100632,-0.672616,0.0166911,-0.975017,-0.948699,0.199991,-1.45986,1
2,0.150101,-0.0949285,-0.184945,-0.0983489,-0.391432,-0.18801,0.0285296,-0.0161905,-0.121014,-0.0820575,0.0289465,-0.241,-0.0497278,-0.113254,0.0144902,-0.0679414,0.091232,-0.0871802,0.224873,-0.360406,-0.0549974,0.14137,-0.152328,0.0963717,0.0146297,0.133607,0.0574333,-0.00477375,0.0388853,-0.0251659,0.0246465,0.0958618,0.0408889,-0.0824225,0.00480227,0.0145226,0.106248,-0.0260799,0.00678029,0.0709676,-0.00145548,0.0368688,-0.0140505,-0.0477347,-0.00236543,0.0286215,0.010334,0.0410246,-0.0044563,0.00943899,-1.08767,-0.646035,1.18604,-1.6944,1.29615,-0.262405,-0.543818,-1.08787,1
3,1.10407,0.667321,-0.7901,0.474833,-0.181566,0.468235,0.641853,-0.304984,-0.0206799,0.0824786,-0.248757,0.154852,-0.0675149,-0.217134,-0.0285596,0.10677,0.089794,-0.110112,0.125948,0.0764637,0.0824017,-0.0261773,0.191051,-0.0978534,-0.0311746,-0.27779,-0.16408,0.0251098,-0.194878,0.183067,-0.117723,0.0377545,-0.00108397,0.0392433,-0.160664,-0.0321161,0.0591017,-0.144108,0.0563911,-0.0330523,-0.100569,-0.0473969,0.126433,0.0622828,0.0789649,0.0337574,-0.0270857,0.0359153,0.0207082,-0.0215894,0.274863,0.1999,0.665614,0.290954,0.673794,1.44826,0.335712,0.26353,1
4,-0.960787,-0.432979,0.0254428,0.00672305,0.511481,-0.384445,-0.0464265,0.314712,-0.273671,0.265535,-0.387132,-0.212078,0.128591,-0.0637984,0.133268,-0.0843437,0.0593618,0.125117,0.0690621,0.0615892,0.123129,-0.112667,0.0598791,0.117148,-0.0240373,0.0296029,-0.0369565,-0.0637915,0.0156372,0.0520545,-0.131558,0.00579464,-0.0696548,0.0906326,0.134287,-0.00432425,0.0243482,-0.100518,0.0855104,0.0299688,0.00172561,-0.0420944,0.0165021,0.0296568,0.0755803,-0.0156707,-0.0147708,0.0479558,-0.0681452,0.0141051,-1.25598,0.367607,-0.0406741,0.162457,-0.631876,1.29601,0.648996,-0.0781037,1
5,0.502045,-0.192927,0.125147,-0.0949673,-0.0443038,0.0168323,0.254674,0.140235,-0.46861,-0.0216542,-0.235253,0.21697,-0.00456953,0.0710191,-0.290749,0.114326,-0.253772,0.117333,-0.0466274,0.0557321,-0.191152,0.116769,-0.0985852,0.0226572,0.0301438,0.00671936,0.0924599,0.0171998,-0.099897,-0.0964279,0.0211018,0.0550999,0.00726871,0.0249526,0.0760152,0.076478,-0.0994178,-0.0526071,0.0824262,0.0824779,0.0185915,-0.0562357,-0.0803507,-0.0336593,0.00916239,0.0069792,-0.0355332,0.0081278,0.00825193,-0.0248457,0.659577,-0.0417959,-0.0964337,0.415895,1.48346,-1.46048,-1.32646,0.558459,1


In [12]:
df

Row,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability,target
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,1.25902,1.07441,-0.0870012,-0.0666432,-0.4028,-0.214531,-0.115619,-0.204249,-0.0865234,-0.126472,-0.0529494,-0.187418,-0.33505,0.22008,0.106203,0.0541629,0.014138,-0.0180455,-0.0715133,-0.0156755,0.179807,-0.190544,0.327443,-0.00827776,0.177347,-0.0582593,-0.0627241,-0.09792,-0.0982399,-0.0377172,-0.0369587,-0.0381895,-0.110458,0.120058,0.00224206,0.235474,0.0601376,-0.0583418,-0.0288259,0.0313048,0.00749979,0.122497,0.0341467,0.144343,0.0757099,0.0698748,-0.0699144,-0.01949,-0.0971503,-0.0809212,-0.374342,0.100632,-0.672616,0.0166911,-0.975017,-0.948699,0.199991,-1.45986,1
2,0.150101,-0.0949285,-0.184945,-0.0983489,-0.391432,-0.18801,0.0285296,-0.0161905,-0.121014,-0.0820575,0.0289465,-0.241,-0.0497278,-0.113254,0.0144902,-0.0679414,0.091232,-0.0871802,0.224873,-0.360406,-0.0549974,0.14137,-0.152328,0.0963717,0.0146297,0.133607,0.0574333,-0.00477375,0.0388853,-0.0251659,0.0246465,0.0958618,0.0408889,-0.0824225,0.00480227,0.0145226,0.106248,-0.0260799,0.00678029,0.0709676,-0.00145548,0.0368688,-0.0140505,-0.0477347,-0.00236543,0.0286215,0.010334,0.0410246,-0.0044563,0.00943899,-1.08767,-0.646035,1.18604,-1.6944,1.29615,-0.262405,-0.543818,-1.08787,1
3,1.10407,0.667321,-0.7901,0.474833,-0.181566,0.468235,0.641853,-0.304984,-0.0206799,0.0824786,-0.248757,0.154852,-0.0675149,-0.217134,-0.0285596,0.10677,0.089794,-0.110112,0.125948,0.0764637,0.0824017,-0.0261773,0.191051,-0.0978534,-0.0311746,-0.27779,-0.16408,0.0251098,-0.194878,0.183067,-0.117723,0.0377545,-0.00108397,0.0392433,-0.160664,-0.0321161,0.0591017,-0.144108,0.0563911,-0.0330523,-0.100569,-0.0473969,0.126433,0.0622828,0.0789649,0.0337574,-0.0270857,0.0359153,0.0207082,-0.0215894,0.274863,0.1999,0.665614,0.290954,0.673794,1.44826,0.335712,0.26353,1
4,-0.960787,-0.432979,0.0254428,0.00672305,0.511481,-0.384445,-0.0464265,0.314712,-0.273671,0.265535,-0.387132,-0.212078,0.128591,-0.0637984,0.133268,-0.0843437,0.0593618,0.125117,0.0690621,0.0615892,0.123129,-0.112667,0.0598791,0.117148,-0.0240373,0.0296029,-0.0369565,-0.0637915,0.0156372,0.0520545,-0.131558,0.00579464,-0.0696548,0.0906326,0.134287,-0.00432425,0.0243482,-0.100518,0.0855104,0.0299688,0.00172561,-0.0420944,0.0165021,0.0296568,0.0755803,-0.0156707,-0.0147708,0.0479558,-0.0681452,0.0141051,-1.25598,0.367607,-0.0406741,0.162457,-0.631876,1.29601,0.648996,-0.0781037,1
5,0.502045,-0.192927,0.125147,-0.0949673,-0.0443038,0.0168323,0.254674,0.140235,-0.46861,-0.0216542,-0.235253,0.21697,-0.00456953,0.0710191,-0.290749,0.114326,-0.253772,0.117333,-0.0466274,0.0557321,-0.191152,0.116769,-0.0985852,0.0226572,0.0301438,0.00671936,0.0924599,0.0171998,-0.099897,-0.0964279,0.0211018,0.0550999,0.00726871,0.0249526,0.0760152,0.076478,-0.0994178,-0.0526071,0.0824262,0.0824779,0.0185915,-0.0562357,-0.0803507,-0.0336593,0.00916239,0.0069792,-0.0355332,0.0081278,0.00825193,-0.0248457,0.659577,-0.0417959,-0.0964337,0.415895,1.48346,-1.46048,-1.32646,0.558459,1
6,0.385327,0.240871,0.0179897,-0.154736,0.0376188,-0.522859,0.288639,0.156529,0.00273436,0.0888874,-0.0449988,0.0367064,0.0299696,-0.0567926,-0.104772,-0.0229654,-0.0353212,-0.0385265,-0.0365305,-0.00352844,-0.0886061,-0.0985073,-0.104116,-0.142604,0.0615494,-0.024266,0.0840438,-0.0588857,-0.0181282,0.055953,0.10269,-0.0358469,0.0144552,0.0187587,-0.0201868,0.04412

In [13]:
# Set random seed for reproducibility
Random.seed!(42)

# Determine the proportion for training (e.g., 80% for training and 20% for testing)
train_proportion = 0.8
n = nrow(df)

# Get the indices for the training set (80% of the data)
train_indices = randperm(n)[1:floor(Int, train_proportion * n)]

# Get the training and testing DataFrames
train_df = df[train_indices, :]
test_df = df[setdiff(1:n, train_indices), :]

# Print the resulting DataFrames
println("Training DataFrame:")
println(train_df)
println("Testing DataFrame:")
println(test_df)

Training DataFrame:
6243×59 DataFrame
  Row │ 0             1             2             3             4             5             6             7             8             9             10            11            12            13            14            15            16            17            18            19            20            21            22            23            24            25            26            27            28            29            30            31            32            33            34            35            36            37            38            39            40            41            42            43            44            45            46            47            48            49            chou_fasman  emini         kolaskar_tongaonkar  parker       isoelectric_point  aromaticity  hydrophobicity  stability    target
      │ Float64       Float64       Float64       Float64       Float64       Float64       Float64       Float64       Float

Excessive output truncated after 589844 bytes.

In [14]:
X_train = train_df[:, Not([:target])]
y_train = train_df[:, :target]

X_test = test_df[:, Not([:target])]
y_test = test_df[:, :target]

1561-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 0
 1
 1
 0
 0
 0
 0
 ⋮
 0
 0
 1
 0
 0
 0
 1
 0
 1
 0
 1
 1

In [15]:
# Train OCT on training data
depths = [10]
minbuckets = [20]

grid_search = IAI.GridSearch(IAI.OptimalTreeClassifier(random_seed=42,criterion=:gini),
    max_depth=depths,
    minbucket=minbuckets)
IAI.fit_cv!(grid_search, X_train, y_train, n_folds=10)
lnr = IAI.get_learner(grid_search)

Fitted OptimalTreeClassifier:
  1) Split: aromaticity < 0.372
    2) Split: aromaticity < -0.2116
      3) Split: 4 < -0.5978
        4) Predict: 1 (82.05%), [7,32], 39 points, error 0.2945
        5) Split: 4 < 0.08758
          6) Split: hydrophobicity < -0.0228
            7) Split: 25 < -0.2151
              8) Predict: 0 (79.17%), [19,5], 24 points, error 0.3299
              9) Split: aromaticity < -1.393
                10) Split: 10 < 0.2477
                  11) Split: stability < 0.2058
                    12) Split: 1 < 0.002381
                      13) Predict: 0 (100.00%), [42,0], 42 points, error 0
                      14) Predict: 0 (80.00%), [16,4], 20 points, error 0.32
                    15) Predict: 1 (76.00%), [6,19], 25 points, error 0.3648
                  16) Predict: 0 (84.00%), [21,4], 25 points, error 0.2688
                17) Split: isoelectric_point < 0.7254
                  18) Split: isoelectric_point < -1.103
                    19) Split: 18 < -0.07707
                      20) Predict: 0 (50.00%), [11,11], 22 points, error 0.5
                      21) Predict: 0 (90.00%), [36,4], 40 points, error 0.18
                    22) Split: emini < -0.5983
                      23) Predict: 1 (66.67%), [8,16], 24 points, error 0.4444
                      24) Predict: 1 (97.60%), [3,122], 125 points, error 0.04685
                  25) Split: emini < -0.531
                    26) Split: 13 < 0.05497
                      27) Predict: 0 (55.10%), [27,22], 49 points, error 0.4948
                      28) Predict: 0 (90.48%), [19,2], 21 points, error 0.1723
                    29) Split: aromaticity < -0.7463
                      30) Predict: 1 (64.52%), [11,20], 31 points, error 0.4579
                      31) Predict: 0 (85.71%), [78,13], 91 points, error 0.2449
            32) Split: hydrophobicity < 1.147
              33) Split: emini < -0.5865
                34) Split: 7 < -0.00207
                  35) Predict: 1 (81.48%), [5,22], 27 points, error 0.3018
                  36) Predict: 0 (71.88%), [23,9], 32 points, error 0.4043
                37) Split: 3 < -0.485
                  38) Predict: 0 (77.78%), [21,6], 27 points, error 0.3457
                  39) Split: 28 < -0.07784
                    40) Split: 42 < -0.003669
                      41) Predict: 1 (69.57%), [7,16], 23 points, error 0.4234
                      42) Predict: 1 (97.50%), [1,39], 40 points, error 0.04875
                    43) Split: isoelectric_point < -0.7538
                      44) Predict: 1 (89.20%), [19,157], 176 points, error 0.1926
                      45) Predict: 1 (55.04%), [58,71], 129 points, error 0.4949
              46) Split: 25 < 0.01593
                47) Predict: 1 (100.00%), [0,69], 69 points, error 0
                48) Predict: 1 (82.86%), [6,29], 35 points, error 0.2841
          49) Split: isoelectric_point < 0.3635
            50) Split: 25 < 0.07478
              51) Split: 30 < 0.03183
                52) Split: aromaticity < -0.675
                  53) Split: 1 < 0.4199
                    54) Split: isoelectric_point < -0.637
                      55) Predict: 1 (71.76%), [24,61], 85 points, error 0.4053
                      56) Predict: 0 (86.49%), [32,5], 37 points, error 0.2337
                    57) Split: 38 < -0.01262
                      58) Predict: 1 (51.28%), [19,20], 39 points, error 0.4997
                      59) Predict: 0 (90.00%), [18,2], 20 points, error 0.18
                  60) Split: isoelectric_point < -1.079
                    61) Predict: 0 (58.97%), [23,16], 39 points, error 0.4839
                    62) Split: 1 < 0.1399
                      63) Predict: 1 (96.72%), [2,59], 61 points, error 0.06342
                      64) Predict: 1 (65.62%), [11,21], 32 points, error 0.4512
                65) Split: 9 < 0.1231
                  66) Split: 48 < 0.03568
                    67) Split: 26 < 0.01995
                      68) Predict: 0 (52.17%

In [16]:
IAI.get_best_params(grid_search)

Dict{Symbol, Any} with 3 entries:
  :minbucket => 20
  :cp        => 0.000226336
  :max_depth => 10

# Report Performance

In [17]:
# Make predictions
y_pred = IAI.predict(lnr, X_test)
y_prob = IAI.predict_proba(lnr, X_test)[:,1]
# Calculate AUC, accuracy for test set 
println("Test AUC score: ", round(IAI.score(lnr,X_test,y_test,criterion=:auc), digits=4))
println("Test Accuracy: ", round(IAI.score(lnr,X_test, y_test,criterion=:accuracy,positive_label=1), digits=4))

# Confusion Matrix (True Positives, False Positives, False Negatives, True Negatives)
TP = sum((y_test .== 1) .& (y_pred .== 1))  # True Positives
FP = sum((y_test .== 0) .& (y_pred .== 1))  # False Positives
FN = sum((y_test .== 1) .& (y_pred .== 0))  # False Negatives
TN = sum((y_test .== 0) .& (y_pred .== 0))  # True Negatives

# Precision
precisions = TP / (TP + FP)
println("Precision: ", precisions)

# Recall
recall = TP / (TP + FN)
println("Recall: ", recall)

# Print confusion matrix
println("Confusion Matrix:")
println("TP: $TP, FP: $FP, FN: $FN, TN: $TN")

# F1 Score
f1 = 2 * (precisions * recall) / (precisions + recall)
println("F1 Score: ", f1)

# # AUC Calculation using ROC (manually with the trapezoidal rule)
# # Sorting probabilities and calculating the ROC curve
# sorted_indices = sortperm(y_prob)
# sorted_y_test = y_test[sorted_indices]
# sorted_y_pred = y_pred[sorted_indices]
# sorted_y_prob = y_prob[sorted_indices]

# # Calculate the true positive rate (TPR) and false positive rate (FPR)
# TPR = cumsum(sorted_y_test) ./ sum(sorted_y_test)  # True Positive Rate (Recall)
# FPR = cumsum(1 .- sorted_y_test) ./ sum(1 .- sorted_y_test)  # False Positive Rate

# # Calculate AUC using the trapezoidal rule
# auc = sum(diff(FPR) .* (TPR[2:end] .+ TPR[1:end-1])) / 2  # Trapezoidal rule
# println("AUC: ", auc)


Test AUC score: 0.7948
Test Accuracy: 0.7239
Precision: 0.7116104868913857
Recall: 0.7402597402597403
Confusion Matrix:
TP: 570, FP: 231, FN: 200, TN: 560
F1 Score: 0.725652450668364


## Test Tabular data only

In [18]:
X_train = X_train[:, Not(1:50)]
X_test = X_test[:, Not(1:50)]

Row,chou_fasman,emini,kolaskar_tongaonkar,parker,isoelectric_point,aromaticity,hydrophobicity,stability
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.374342,0.100632,-0.672616,0.0166911,-0.975017,-0.948699,0.199991,-1.45986
2,0.659577,-0.0417959,-0.0964337,0.415895,1.48346,-1.46048,-1.32646,0.558459
3,1.51717,-0.631854,-1.28597,0.30619,1.5214,-0.985134,-3.63331,2.1464
4,0.17067,-0.588694,0.368229,0.490556,1.76152,-0.954353,-0.0277462,0.255012
5,-0.51861,0.666027,1.07452,-0.623255,-1.10512,0.688146,-0.1906,0.738504
6,2.1263,0.866412,-2.6242,1.66227,1.14342,-0.678593,0.657629,-0.52261
7,-0.89531,-0.112701,-0.245126,-1.15502,1.11214,0.656268,-0.269871,-0.167996
8,1.89387,-0.33775,-1.30456,0.367138,-0.805353,0.29521,0.178164,-0.0992261
9,0.0985356,-0.129965,-0.393818,-0.211861,-0.916071,0.765251,0.0731701,-0.289585


In [20]:
# Train OCT on training data
depths = 10
minbuckets = [20,50,100]
cpi=[0.01,0.1]

grid_search = IAI.GridSearch(IAI.OptimalTreeClassifier(random_seed=42,criterion=:gini),
    max_depth=depths,
    minbucket=minbuckets,
    cp=cpi
)
IAI.fit_cv!(grid_search, X_train, y_train, n_folds=10)
lnr = IAI.get_learner(grid_search)

Fitted OptimalTreeClassifier:
  1) Split: hydrophobicity < 0.2112
    2) Split: isoelectric_point < -0.6642
      3) Split: aromaticity < 2.034
        4) Split: hydrophobicity < -0.5824
          5) Predict: 0 (56.97%), [229,173], 402 points, error 0.4903
          6) Predict: 1 (79.32%), [122,468], 590 points, error 0.328
        7) Predict: 0 (90.62%), [87,9], 96 points, error 0.1699
      8) Split: aromaticity < 0.2692
        9) Split: stability < 0.1922
          10) Predict: 0 (75.15%), [620,205], 825 points, error 0.3735
          11) Split: isoelectric_point < 1.061
            12) Predict: 1 (80.00%), [38,152], 190 points, error 0.32
            13) Predict: 0 (67.26%), [341,166], 507 points, error 0.4404
        14) Split: stability < -0.5928
          15) Predict: 0 (60.30%), [161,106], 267 points, error 0.4788
          16) Predict: 1 (84.10%), [45,238], 283 points, error 0.2675
    17) Split: isoelectric_point < 0.5991
      18) Split: hydrophobicity < 0.2657
        19) Predict: 0 (87.95%), [387,53], 440 points, error 0.2119
        20) Predict: 1 (72.17%), [460,1193], 1653 points, error 0.4017
      21) Split: aromaticity < 0.3095
        22) Split: aromaticity < -0.4471
          23) Predict: 1 (82.80%), [27,130], 157 points, error 0.2848
          24) Predict: 0 (82.32%), [461,99], 560 points, error 0.2911
        25) Predict: 1 (51.28%), [133,140], 273 points, error 0.4997

In [21]:
IAI.get_best_params(grid_search)

Dict{Symbol, Any} with 3 entries:
  :minbucket => 20
  :cp        => 0.01
  :max_depth => 10

In [22]:
# Make predictions
y_pred = IAI.predict(lnr, X_test)
y_prob = IAI.predict_proba(lnr, X_test)[:,1]
# Calculate AUC, accuracy for test set 
println("Test AUC score: ", round(IAI.score(lnr,X_test,y_test,criterion=:auc), digits=4))
println("Test Accuracy: ", round(IAI.score(lnr,X_test, y_test,criterion=:accuracy,positive_label=1), digits=4))

# Confusion Matrix (True Positives, False Positives, False Negatives, True Negatives)
TP = sum((y_test .== 1) .& (y_pred .== 1))  # True Positives
FP = sum((y_test .== 0) .& (y_pred .== 1))  # False Positives
FN = sum((y_test .== 1) .& (y_pred .== 0))  # False Negatives
TN = sum((y_test .== 0) .& (y_pred .== 0))  # True Negatives

# Precision
precisions = TP / (TP + FP)
println("Precision: ", precisions)

# Recall
recall = TP / (TP + FN)
println("Recall: ", recall)

# Print confusion matrix
println("Confusion Matrix:")
println("TP: $TP, FP: $FP, FN: $FN, TN: $TN")

# F1 Score
f1 = 2 * (precisions * recall) / (precisions + recall)
println("F1 Score: ", f1)

# # AUC Calculation using ROC (manually with the trapezoidal rule)
# # Sorting probabilities and calculating the ROC curve
# sorted_indices = sortperm(y_prob)
# sorted_y_test = y_test[sorted_indices]
# sorted_y_pred = y_pred[sorted_indices]
# sorted_y_prob = y_prob[sorted_indices]

# # Calculate the true positive rate (TPR) and false positive rate (FPR)
# TPR = cumsum(sorted_y_test) ./ sum(sorted_y_test)  # True Positive Rate (Recall)
# FPR = cumsum(1 .- sorted_y_test) ./ sum(1 .- sorted_y_test)  # False Positive Rate

# # Calculate AUC using the trapezoidal rule
# auc = sum(diff(FPR) .* (TPR[2:end] .+ TPR[1:end-1])) / 2  # Trapezoidal rule
# println("AUC: ", auc)


Test AUC score: 0.7677
Test Accuracy: 0.7162
Precision: 0.7148488830486203
Recall: 0.7064935064935065
Confusion Matrix:
TP: 544, FP: 217, FN: 226, TN: 574
F1 Score: 0.7106466361854998
